<a href="https://colab.research.google.com/github/Blacksujit/Deep-Learning-Specialization-Repo/blob/main/New_AI_doctor_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install transformers spacy gradio fpdf langchain_community langchain_huggingface
!pip install transformers spacy langchain gradio fpdf
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
import spacy
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.llms import HuggingFacePipeline
import random
import json
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
# Load SpaCy model for NER
nlp = spacy.load("en_core_web_sm")

# Load Hugging Face pipelines

intent_classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")
bio_bert_model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
bio_bert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
treatment_advice_model = pipeline("text-generation", model="gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [4]:
# Load the knowledge base
try:
    with open("/content/conditions.json", "r") as f:
        knowledge_base = json.load(f)["common_conditions"]
except FileNotFoundError:
    logger.error("Error: The JSON file was not found. Please check the file path.")
    knowledge_base = []
except json.JSONDecodeError as e:
    logger.error(f"Error: Failed to decode JSON. Details: {e}")
    knowledge_base = []

In [5]:
# Generate empathetic response
def generate_empathetic_response():
    responses = [
        "I'm here to help. Please tell me more about your symptoms.",
        "I understand this can be concerning. Let me assist you.",
        "Your health is important. Let's work together to address your concerns."
    ]
    return random.choice(responses)

# Intent Detection Agent
def intent_detection_agent(user_input):
    """Detect the intent of the user input."""
    try:
        intent_result = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")(user_input)
        return intent_result[0]["label"]
    except Exception as e:
        logger.error(f"Error in intent detection: {e}")
        return "unknown"

# Entity Recognition Agent
def entity_recognition_agent(user_input):
    """Extract entities such as symptoms or conditions from the user's input."""
    try:
        doc = nlp(user_input)
        entities = {"symptoms": [ent.text for ent in doc.ents if ent.label_ == "SYMPTOM"]}
        return entities
    except Exception as e:
        logger.error(f"Error in entity recognition: {e}")
        return {"symptoms": []}

# Diagnostic Reasoning Agent
def diagnostic_reasoning_agent(entities):
    """Perform diagnostic reasoning based on extracted entities."""
    try:
        matched_conditions = []
        for condition in knowledge_base:
            for symptom in condition["symptoms"]:
                if symptom.lower() in [entity.lower() for entity in entities.get("symptoms", [])]:
                    matched_conditions.append(condition)
                    break

        if matched_conditions:
            response = generate_empathetic_response()
            for condition in matched_conditions:
                response += f"\n\nCondition: {condition['name']}\nAdvice: {condition['advice']}"
            return response
        else:
            return "I'm sorry, I couldn't find specific advice for your query. Please consult a medical professional."
    except Exception as e:
        logger.error(f"Error in diagnostic reasoning: {e}")
        return "Error in diagnostic reasoning."

# Treatment Recommendation Agent
def treatment_recommendation_agent(entities):
    """Provide treatment recommendations based on extracted entities."""
    try:
        recommendations = []
        for condition in knowledge_base:
            for symptom in condition["symptoms"]:
                if symptom.lower() in [entity.lower() for entity in entities.get("symptoms", [])]:
                    recommendations.append(f"For {condition['name']}: {condition['advice']}")
                    break
        return recommendations
    except Exception as e:
        logger.error(f"Error in treatment recommendation: {e}")
        return []

# Safety Compliance Agent
def safety_compliance_agent(entities):
    """Check for critical conditions and provide safety advice."""
    try:
        critical_conditions = ["chest pain", "shortness of breath"]
        for symptom in entities.get("symptoms", []):
            if symptom.lower() in critical_conditions:
                return "⚠️ Critical condition detected! Seek immediate medical attention."
        return "✔️ No immediate critical conditions detected."
    except Exception as e:
        logger.error(f"Error in safety compliance check: {e}")
        return "Error in safety compliance check."

In [6]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

def create_fusion_agentic_pipeline():
    """Create a robust fusion agentic pipeline."""

    # Explicitly specify the tokenizer and model
    model_name = "gpt2"  # Replace with a medical-specific model if available
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Initialize Hugging Face pipeline with updated parameters
    llm_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=150,  # Generate up to 150 tokens
        temperature=0.7,  # Adjust creativity
        pad_token_id=tokenizer.eos_token_id  # Ensure proper padding
    )
    llm = HuggingFacePipeline(pipeline=llm_pipeline)

    # Define tools for each agent
    tools = [
        Tool(
            name="Intent Detection",
            func=intent_detection_agent,
            description="Detects the user's intent from their input."
        ),
        Tool(
            name="Entity Recognition",
            func=entity_recognition_agent,
            description="Extracts entities such as symptoms or conditions from the user's input."
        ),
        Tool(
            name="Diagnostic Reasoning",
            func=diagnostic_reasoning_agent,
            description="Performs diagnostic reasoning based on extracted entities."
        ),
        Tool(
            name="Treatment Recommendation",
            func=treatment_recommendation_agent,
            description="Provides treatment recommendations based on extracted entities."
        ),
        Tool(
            name="Safety Compliance Check",
            func=safety_compliance_agent,
            description="Checks for critical conditions and provides safety advice."
        )
    ]

    # Initialize the agent
    agent = initialize_agent(
        tools=tools,
        llm=llm,  # Pass the Hugging Face pipeline as the LLM
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True  # Retry on parsing errors
    )
    return agent

In [7]:
# # Process user input through the agentic pipeline
# def process_user_input(user_input):
#     """Process user input through the agentic pipeline."""
#     intent = intent_detection_agent(user_input)
#     entities = entity_recognition_agent(user_input)
#     diagnostic_response = diagnostic_reasoning_agent(entities)
#     treatment_recommendations = treatment_recommendation_agent(entities)
#     safety_check = safety_compliance_agent(entities)

#     return {
#         "Intent": intent,
#         "Entities": entities,
#         "Diagnostic Response": diagnostic_response,
#         "Treatment Recommendations": treatment_recommendations,
#         "Safety Check": safety_check
#     }

# # Create a Gradio interface for user interaction
# def create_gradio_interface():
#     """Create a Gradio interface for user interaction."""
#     return gr.Interface(
#         fn=process_user_input,
#         inputs=gr.Textbox(lines=2, placeholder="Enter your symptoms or health concerns here..."),
#         outputs=[
#             gr.Textbox(label="Detected Intent"),
#             gr.JSON(label="Extracted Entities"),
#             gr.Textbox(label="Diagnostic Response"),
#             gr.JSON(label="Treatment Recommendations"),
#             gr.Textbox(label="Safety Check")
#         ],
#         title="AI Doctor Chatbot",
#         description="Interact with the AI Doctor Chatbot to get diagnostic reasoning, treatment recommendations, and safety compliance checks."
#     )

In [8]:
# # Launch the Gradio interface
# if __name__ == "__main__":
#     interface = create_gradio_interface()
#     interface.launch()

In [9]:
if __name__ == "__main__":
    # Initialize the pipeline
    try:
        agent = create_fusion_agentic_pipeline()
        print("Pipeline initialized successfully.")
    except Exception as e:
        print(f"Error during pipeline initialization: {e}")
        exit()

    # Test the pipeline with a sample input
    user_input = "I have a headache and chest pain"
    print(f"\nUser Input: {user_input}")

    try:
        response = agent.invoke(user_input)  # Use `invoke` instead of `run` (updated method)
        print("\nPipeline Response:")
        print(response)
    except Exception as e:
        print(f"Error during pipeline execution: {e}")

Device set to use cuda:0
<ipython-input-6-cedb6e5dea3f>:53: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


Pipeline initialized successfully.

User Input: I have a headache and chest pain


> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Intent Detection(user_input) - Detects the user's intent from their input.
Entity Recognition(user_input) - Extracts entities such as symptoms or conditions from the user's input.
Diagnostic Reasoning(entities) - Performs diagnostic reasoning based on extracted entities.
Treatment Recommendation(entities) - Provides treatment recommendations based on extracted entities.
Safety Compliance Check(entities) - Checks for critical conditions and provides safety advice.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Intent Detection, Entity Recognition, Diagnostic Reasoning, Treatment Recommend

Token indices sequence length is longer than the specified maximum sequence length for this model (1683 > 1024). Running this sequence through the model will result in indexing errors


Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

Intent Detection(user_input) - Detects the user's intent from their input.
Entity Recognition(user_input) - Extracts entities such as symptoms or conditions from the user's input.
Diagnostic Reasoning(entities) - Performs diagnostic reasoning based on extracted entities.
Treatment Recommendation(entities) - Provides treatment recommendations based on extracted entities.
Safety Compliance Check(entities) - Checks for critical conditions and provides safety advice.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Intent Detection, Entity Recognition, Diagnostic Reasoning, Treatment Recommendation, Safety Compliance Check]
Action Input: the input to the action
Observation: the result of the action
... (this Tho

In [10]:
!pip install -U langchain langchain-openai faiss-cpu gradio langchain-community python-dotenv duckduckgo-search
!pip install -U langchain-anthropic faiss-cpu gradio langchain-core langchain-community python-dotenv duckduckgo-search
# Step 1: Install all dependencies
!pip install -U langchain-anthropic gradio faiss-cpu duckduckgo-search huggingface-hub ratelimit python-dotenv > /dev/null


In [11]:
# # Step 2: Upload your medical knowledge file
# from google.colab import files
# uploaded = files.upload()
# kb_filename = next(iter(uploaded))  # Get uploaded filename


In [12]:
# Step 2: Import libraries with updated syntax
import os
import time
from datetime import datetime
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry
from google.colab import files
import gradio as gr

# Updated LangChain imports
from langchain_anthropic import ChatAnthropic
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.messages import AIMessage, HumanMessage

In [13]:
# Step 3: Configuration
load_dotenv()

class Config:
    RATE_LIMIT = 25  # Calls per minute
    MAX_TOKENS = 4000
    DAILY_LIMIT = 500
    MODEL_NAME = "claude-3-haiku-20240307"

config = Config()

In [14]:
# Step 4: Rate Limiter with Enhanced Tracking
class APIManager:
    def __init__(self):
        self.call_count = 0
        self.start_time = time.time()
        self.daily_count = 0
        self.last_reset = datetime.now().date()

    def check_daily_limit(self):
        today = datetime.now().date()
        if today != self.last_reset:
            self.daily_count = 0
            self.last_reset = today
        return self.daily_count >= config.DAILY_LIMIT

    @sleep_and_retry
    @limits(calls=config.RATE_LIMIT, period=60)
    def call_api(self, llm, prompt):
        if self.check_daily_limit():
            raise Exception("Daily limit reached")

        self.call_count += 1
        self.daily_count += 1
        return llm.invoke(prompt)

api_manager = APIManager()


In [15]:
# Step 5: Initialize Claude with Safety Limits
llm = ChatAnthropic(
    model_name=config.MODEL_NAME,
    temperature=0.3,
    max_tokens=config.MAX_TOKENS
)

In [16]:
# Step 6: Knowledge Base Setup
def load_knowledge_base():
    print("Please upload your medical knowledge file:")
    uploaded = files.upload()
    kb_filename = next(iter(uploaded))

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    loader = TextLoader(kb_filename)
    docs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(loader.load())

    return FAISS.from_documents(docs, embeddings).as_retriever(search_kwargs={"k": 3})

retriever = load_knowledge_base()

Please upload your medical knowledge file:


Saving conditions.json to conditions (1).json


<ipython-input-16-f0c1edef7055>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Step 7: Medical Tools with Rate Limiting (updated tool definition)
@tool
def knowledge_lookup(query: str) -> str:
    """Search medical knowledge base"""
    try:
        docs = retriever.get_relevant_documents(query)
        prompt = f"Summarize this medical information concisely:\n\n{docs}"
        response = api_manager.call_api(llm, prompt)
        return response.content
    except Exception as e:
        return f"⚠️ System busy. Please try again later. Error: {str(e)}"

@tool
def symptom_triage(symptoms: str) -> str:
    """Assess symptom urgency"""
    try:
        prompt = f"""Analyze these symptoms: {symptoms}. Respond with:
        1. Urgency level (Emergency/Urgent/Routine)
        2. 3 possible conditions
        3. Recommended next steps"""
        return api_manager.call_api(llm, prompt).content
    except Exception as e:
        return str(e)

tools = [knowledge_lookup, symptom_triage]

In [ ]:
# Step 8: Create Agent with Memory (updated syntax)
medical_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are DoctorAI, a cautious medical assistant. Rules:
1. First determine urgency using symptom_triage
2. Use knowledge_lookup for medical facts
3. Never diagnose - suggest possibilities only
4. Always recommend doctor consultation
5. Keep responses under 300 words"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

llm_with_tools = llm.bind_tools(tools)

agent = {
    "input": lambda x: x["input"],
    "chat_history": lambda x: x["chat_history"],
    "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
} | medical_prompt | llm_with_tools | OpenAIToolsAgentOutputParser()

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
# Step 9: Gradio Interface with Usage Stats
def get_usage_stats():
    return (
        f"📊 Usage: {api_manager.daily_count}/{config.DAILY_LIMIT} (daily) | "
        f"{api_manager.call_count}/{config.RATE_LIMIT} (minute)"
    )

def respond(message, history):
    # Convert Gradio history to LangChain format
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))

    # Input validation
    if len(message) > 500:
        return "❌ Query too long (max 500 characters)", history

    try:
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history
        })

        formatted = (
            f"{response['output']}\n\n"
            f"{get_usage_stats()}\n"
            "⚠️ Not medical advice - consult a real doctor"
        )
        history.append((message, formatted))
        return "", history
    except Exception as e:
        return f"🚨 System error: {str(e)}", history

with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🏥 AI Medical Assistant (Claude 3)")

    with gr.Row():
        chatbot = gr.Chatbot(height=450)
        with gr.Column(scale=0.3):
            gr.Markdown("### Safety Info")
            gr.HTML("""
            <ul style="color: #555;">
                <li>For emergencies, call local emergency services</li>
                <li>This is an AI assistant only</li>
                <li>Always verify with your doctor</li>
            </ul>
            """)
            usage_stats = gr.Label(get_usage_stats(), label="API Usage")

    msg = gr.Textbox(label="Health Concern", placeholder="Describe symptoms...")
    clear_btn = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # Auto-update usage stats
    app.load(
        fn=lambda: usage_stats.update(value=get_usage_stats()),
        inputs=None,
        outputs=usage_stats,
        every=5
    )

# Launch with sharing enabled
app.launch(share=True, debug=True)

In [ ]:
# Step 1: Install dependencies
!pip install -U langchain-anthropic gradio faiss-cpu duckduckgo-search huggingface-hub ratelimit python-dotenv > /dev/null

# Step 2: Import libraries
import os
import time
from datetime import datetime
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry
from google.colab import files
import gradio as gr

# LangChain components
from langchain_anthropic import ChatAnthropic
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import AIMessage, HumanMessage
from langchain.tools import tool

# Step 3: Configuration
load_dotenv()

class Config:
    RATE_LIMIT = 25  # Calls per minute
    MAX_TOKENS = 4000
    DAILY_LIMIT = 500
    MODEL_NAME = "claude-3-haiku-20240307"

config = Config()

# Step 4: Rate Limiter
class APIManager:
    def __init__(self):
        self.call_count = 0
        self.start_time = time.time()
        self.daily_count = 0
        self.last_reset = datetime.now().date()

    @sleep_and_retry
    @limits(calls=config.RATE_LIMIT, period=60)
    def call_api(self, llm, prompt):
        if self.check_daily_limit():
            raise Exception("Daily limit reached")
        self.call_count += 1
        self.daily_count += 1
        return llm.invoke(prompt)

    def check_daily_limit(self):
        today = datetime.now().date()
        if today != self.last_reset:
            self.daily_count = 0
            self.last_reset = today
        return self.daily_count >= config.DAILY_LIMIT

api_manager = APIManager()

# Step 5: Initialize Claude
llm = ChatAnthropic(
    model_name=config.MODEL_NAME,
    temperature=0.3,
    max_tokens=config.MAX_TOKENS
)

# Step 6: Knowledge Base Setup
print("Upload your medical knowledge file:")
uploaded = files.upload()
kb_filename = next(iter(uploaded))

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
loader = TextLoader(kb_filename)
docs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(loader.load())
retriever = FAISS.from_documents(docs, embeddings).as_retriever(search_kwargs={"k": 3})

# Step 7: Medical Tools
@tool
def knowledge_lookup(query: str) -> str:
    """Search medical knowledge base"""
    try:
        docs = retriever.get_relevant_documents(query)
        prompt = f"Summarize this medical information:\n\n{docs}"
        return api_manager.call_api(llm, prompt).content
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

@tool
def symptom_triage(symptoms: str) -> str:
    """Assess symptom urgency"""
    try:
        prompt = f"""Analyze symptoms: {symptoms}. Respond with:
        1. Urgency level
        2. Possible conditions
        3. Next steps"""
        return api_manager.call_api(llm, prompt).content
    except Exception as e:
        return str(e)

tools = [knowledge_lookup, symptom_triage]

# Step 8: Create Agent (FIXED PROMPT STRUCTURE)
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are DoctorAI. Follow these rules:
1. First check symptom urgency
2. Use knowledge base for facts
3. Never diagnose
4. Recommend doctor consultation
5. Keep responses concise

Tools: {tools}"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Step 9: Gradio Interface
def get_usage_stats():
    return f"Requests: {api_manager.daily_count}/{config.DAILY_LIMIT} (Daily)"

def respond(message, chat_history):
    # Convert history format
    lc_messages = []
    for msg in chat_history:
        if msg["role"] == "user":
            lc_messages.append(HumanMessage(content=msg["content"]))
        else:
            lc_messages.append(AIMessage(content=msg["content"]))

    try:
        response = agent_executor.invoke({
            "input": message,
            "chat_history": lc_messages,
            "tools": [tool.name for tool in tools]
        })

        return chat_history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": f"{response['output']}\n\n{get_usage_stats()}\n⚠️ Not medical advice"}
        ]
    except Exception as e:
        return chat_history + [{"role": "assistant", "content": f"Error: {str(e)}"}]

# Build interface
with gr.Blocks() as app:
    gr.Markdown("# 🏥 AI Medical Assistant")
    chatbot = gr.Chatbot(height=400, type="messages")
    msg = gr.Textbox(label="Your health concern")
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [chatbot])

    # Auto-update usage
    def update_usage():
        while True:
            time.sleep(5)
            chatbot[-1][1]["content"] = chatbot[-1][1]["content"].split("⚠️")[0] + f"\n\n{get_usage_stats()}\n⚠️ Not medical advice"

    import threading
    threading.Thread(target=update_usage, daemon=True).start()

app.launch(share=True)

In [4]:
# Step 1: Clean environment setup
!pip install --upgrade pip
!pip install --force-reinstall numpy==1.26.0  # Fix numpy version
!pip install gradio sentence-transformers==2.2.2 faiss-cpu==1.7.4 --no-deps
!pip install --no-cache-dir sentence-transformers  # Reinstall with correct dependencies

# Step 2: Verify installations
import sys
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "check"])

# Step 3: Import libraries
from sentence_transformers import SentenceTransformer
from google.colab import files
import gradio as gr
import numpy as np
import os

# Force CPU-only mode
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Step 4: Load lightweight model
print("Loading ultra-lightweight model...")
try:
    embeddings = SentenceTransformer('paraphrase-MiniLM-L3-v2', device='cpu')  # Even smaller than MiniLM-L6
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    # Fallback to dummy embeddings
    class DummyEmbedder:
        def encode(self, text):
            return np.random.rand(384)  # Same dimension as MiniLM
    embeddings = DummyEmbedder()

# Step 5: Knowledge Base Setup
print("Please upload your medical knowledge text file:")
uploaded = files.upload()
kb_filename = next(iter(uploaded))

# Simple text processing
with open(kb_filename, 'r', encoding='utf-8') as f:
    text = f.read()
chunks = [text[i:i+300] for i in range(0, min(5000, len(text)), 300)]  # Use first 5000 chars
print(f"Created {len(chunks)} knowledge chunks")

# Create simple similarity search
embeddings_array = np.array([embeddings.encode(chunk) for chunk in chunks])
index = np.dot(embeddings_array, embeddings_array.T)  # Simple dot product similarity

# Step 6: Response System
MEDICAL_RESPONSES = {
    "emergency": "🆘 EMERGENCY: Please call local emergency services immediately",
    "pain": "🩹 For pain: Rest, OTC pain relievers, see doctor if severe",
    "fever": "🤒 Fever care: Rest, fluids, fever reducers. See doctor if >103°F",
    "default": "💡 I recommend consulting a doctor about your symptoms"
}

def get_response(query):
    """Ultra-lightweight response system"""
    query = query.lower()

    # Emergency check
    emergencies = ["chest pain", "can't breathe", "unconscious", "severe bleeding"]
    if any(term in query for term in emergencies):
        return MEDICAL_RESPONSES["emergency"]

    # Simple pattern matching
    if "pain" in query:
        return MEDICAL_RESPONSES["pain"]
    if "fever" in query:
        return MEDICAL_RESPONSES["fever"]

    # Knowledge lookup
    query_embed = embeddings.encode(query)
    scores = np.dot(embeddings_array, query_embed)
    best_match = chunks[np.argmax(scores)][:200]  # Get top match

    return f"ℹ️ Medical info: {best_match}...\n\n⚠️ Always consult a doctor"

# Step 7: Gradio Interface
with gr.Blocks(title="Lightweight Medical Helper") as app:
    gr.Markdown("## Basic Symptom Checker")
    chatbot = gr.Chatbot(height=200)
    msg = gr.Textbox(label="Describe symptoms", placeholder="e.g. headache, fever...")
    msg.submit(lambda m,h: (h + [(m, get_response(m))], [msg, chatbot], [chatbot]))
    clear = gr.Button("Clear")
    clear.click(lambda: [], None, chatbot)

app.launch(share=True, debug=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.23 requires numpy>=1.26.2; python_version < "3.13", but you have numpy 1.26.0 which is inc

CalledProcessError: Command '['/usr/bin/python3', '-m', 'pip', 'check']' returned non-zero exit status 2.